# 집값 예측

- Data fields
    - ID : 집을 구분하는 번호
    - date : 집을 구매한 날짜
    - price : 집의 가격(Target variable)
    - bedrooms : 침실의 수
    - bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
    - sqft_living : 주거 공간의 평방 피트(면적)
    - sqft_lot : 부지의 평방 피트(면적)
    - floors : 집의 층 수
    - waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
    - view : 집이 얼마나 좋아 보이는지의 정도
    - condition : 집의 전반적인 상태   #건물 연식이 몇년이 지나면 가격 같음
    - grade : King County grading 시스템 기준으로 매긴 집의 등급   
    - sqft_above : 지하실을 제외한 평방 피트(면적)
    - sqft_basement : 지하실의 평방 피트(면적)   # 지하실 있는 데이터는 따로 취급?
    - yr_built : 지어진 년도
    - yr_renovated : 집을 재건축한 년도
    - zipcode : 우편번호
    - lat : 위도
    - long : 경도
    - sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
    - sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)

In [2]:
import pandas as pd
import numpy as np
import random

In [160]:
house = pd.read_csv("dataset/train.csv")

In [161]:
# train 셋을 추출해서 home으로 만들어줍니다.
house.head()  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2,2,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
3,3,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
4,4,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711


In [162]:
# 기본 전처리
#id 컬럼 삭제  #id가 정보 식별하려면 계속 따라다녀야할거같아
# house.drop(columns='id', inplace=True) 

In [163]:
# date(구매시기) 데이터 줄여주기 ex) 20141013T000000 -> 20141013 ->2014.1013  (구매와 건축 시기 차이 구할 때 유용)
date_2 = [round(int(house['date'].iloc[i][:8])/10000, 4) for i in range(len(house))]
house['date'] = date_2

In [164]:
# price 정규화 0 < price < 1
house['price'] = house['price']/7.700000e+06

- Lable : price : 집의 가격(Target variable)  # 최댓값으로 나눠 정규화함
- 시기적 요소 : 건축 년도, 재건축년도, 구매 날짜 (리만브라더스 등 큰 사건에 따른 영향 고려)
    - date : 집을 구매한 날짜 # T0000.. 없애기 #20140502 ~ 20150514
    - yr_built : 지어진 년도
    - yr_renovated : 집을 재건축한 년도  # 재건축은 시기가 또 달라짐
        - 건물 감가상각 (30~50년 : 대략 40년), 그 이후는 토지가격.
        - 건물가 감가상각고려해서 피팅
- 내적 요소 : 집의 면적, 방 갯수 등
    - bedrooms : 침실의 수 
    - bathrooms : 침실 개수 당 화장실의 수(화장실의 수 / 침실의 수 )
    - floors : 집의 층 수  #1~3.5층까지 있다. 아 단독주택들이구나
    - sqft_above : 지하실을 제외한 평방 피트(면적)
    - sqft_basement : 지하실의 평방 피트(면적)   # 지하실 있는 데이터는 따로 취급?
    - sqft_living : 주거 공간의 평방 피트(면적)
    - sqft_lot : 부지의 평방 피트(면적)   #일단 집의 종류를 분류해야할거같은데
    - sqft_living15 : 2015년 기준 주거 공간의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
    - sqft_lot15 : 2015년 기준 부지의 평방 피트(면적, 집을 재건축했다면, 변화가 있을 수 있음)
- 외적 요소 : 뷰, 보이는 상태
    - waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)  # 거의 안보임
    - view : 집이 얼마나 좋아 보이는지의 정도  #0~4점 보통 0점
    - condition : 집의 전반적인 상태   #건물 연식이 몇년이 지나면 가격 같음
- 위치 요소
    - zipcode : 우편번호 #집 위치 판단할때좋을듯. 같은 위치의 분류로 나누기  - 위치와 무관. 삭제
    - lat : 위도  #같은 위치의 분류로 나누기   - 비지도 학습   위치 분류
    - long : 경도  #같은 위치의 분류로 나누기  - 비지도 학습
- 기타 요소
    - grade : King County grading 시스템 기준으로 매긴 집의 등급   
- 영향이 있을까?
    

In [195]:
# (데이터 수가 작아서 전처리 단계에서는 샘플 안해도 될듯)
# 이중 1000개의 데이터를 임의로 샘플 추출 합니다.  
def rd_data(data, number = 100):
    rd_idx = [random.randint(0,len(data)) for _ in range(number)]   # 0 ~ len(data) 개까지의 인덱스 중 랜덤 샘플 # 개 추출
    sample = data.iloc[rd_idx].reset_index(drop=True)  # 샘플 추출
    return sample

### 1. zipcode : 삭제
- zipcode는 위치와 상관이 없다.
- 같은 zipcode를 가지는 집들 중 (위도, 경도)를 구글 지도에 찍어보면 서로 완전 다른 지역을 나타내고 있다.
- 따라서 zipcode는 무의미한 정보이다.

In [196]:
# zipcode와 (위도, 경도) 정보 조사
house[['id', 'price', 'zipcode', 'lat', 'long']].sort_values(by = ['zipcode']).reset_index(drop=True).head()

,id,price,zipcode,lat,long
0,9015,0.037662,98001,47.3530,-122.294
1,13162,0.059740,98001,47.2619,-122.271
2,13160,0.033117,98001,47.3318,-122.277
3,14135,0.053896,98001,47.3359,-122.257
4,1557,0.043117,98001,47.3524,-122.285


In [197]:
#zipcode 삭제
house.drop(columns=['zipcode'], inplace=True)

### 2. 15년 이전 면적 기준, 15년 이후 면적 기준
- 14년 구매엔 이전 것이 적용.
- 15년 구매시엔 15년 것이 적용. 아마도?
- 상관계수를 구해보자.
    - 14년도, 15년 초(몇월까지 ?)에 거래된 집의 가격과 15년 이전 기준 넓이의 상관계수
    - 15년 초 이후에 거래된 집의 가격과 15년 기준 넓이의 상관계수
- 15년 기준과 그 이전 기준 중 고르라고 하면 이전 기준으로 해야할 것 같다.
- 아무래도 부지 넓이랑 가격과는 상관이 없는 것 아닌가 싶다.

In [207]:
house[['id', 'price', 'date', 'sqft_living', 'sqft_living15', 'sqft_lot', 'sqft_lot15', 'yr_renovated']].head()

,id,price,date,sqft_living,sqft_living15,sqft_lot,sqft_lot15,yr_renovated
0,0,0.028818,2014.1013,1180,1340,5650,5650,0
1,1,0.023377,2015.0225,770,2720,10000,8062,0
2,2,0.066234,2015.0218,1680,1800,8080,7503,0
3,3,0.033442,2014.0627,1715,2238,6819,6819,0
4,4,0.037903,2015.0115,1060,1650,9711,9711,0


In [226]:
# 2015이전과 이후로 데이터 구분
criteria = 2015
h_2014 = house[house['date']<criteria]  #2014년도 데이터
h_2015 = house[house['date']>=criteria] #2015년도 데이터

In [239]:
# 주거 공간 넓이랑 가격의 상관계수
h1 = [round(np.corrcoef(h_2014["price"], h_2014["sqft_living"])[0][1], 4), \
round(np.corrcoef(h_2014["price"], h_2014["sqft_living15"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_living"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_living15"])[0][1], 4)]    #상관계수 list ver.
h1_a = np.reshape(np.array(h1),(2,2))  # list to array
pd.DataFrame(h1_a, index=['price 2014', 'price 2015'], columns=["sqft_living", "sqft_living15"]) #DF로 만들기
# 2015 : (0.7149, 0.5851, 0.6772, 0.5899)
# 2015.02 : (0.7133, 0.5899, 0.6762, 0.578)
# 만약 15년 기준과 그 이전 기준 중 고르라고 하면 이전 기준으로 해야할 것 같다.

,sqft_living,sqft_living15
price 2014,0.7149,0.5851
price 2015,0.6772,0.5899


In [240]:
# 부지 넓이랑 가격의 상관계수
h2 = [round(np.corrcoef(h_2014["price"], h_2014["sqft_lot"])[0][1], 4), \
round(np.corrcoef(h_2014["price"], h_2014["sqft_lot15"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_lot"])[0][1], 4), \
round(np.corrcoef(h_2015["price"], h_2015["sqft_lot15"])[0][1], 4)]   #상관계수 list ver.
h2_a = np.reshape(np.array(h2),(2,2))   # list to array
pd.DataFrame(h2_a, index=['price 2014', 'price 2015'], columns=["sqft_lot", "sqft_lot15"]) #DF로 만들기
# (0.0899, 0.0815, 0.1106, 0.0961)
# 아무래도 부지 넓이랑 가격과는 상관이 없는 것 아닌가 싶다.

,sqft_lot,sqft_lot15
price 2014,0.0899,0.0815
price 2015,0.1106,0.0961


### \#. 재건축 여부
- 재건축한 것과 안한 것은 서로 분류해서 생각해야한다.

In [199]:
# 재건축한 것(house_re)과 안한 것(house_no)으로 나누기
house_no = house[house['yr_renovated']==0].reset_index(drop=True)
house_re = house[house['yr_renovated']!=0].reset_index(drop=True)

### \#. 가격정보

In [126]:
h_yr = house[['price', 'date', 'yr_built', 'yr_renovated']]
h_yr.tail()

,price,date,yr_built,yr_renovated
15030,0.079310,2014.1014,2014,0
15031,0.130844,2015.0326,2009,0
15032,0.046753,2014.0521,2009,0
15033,0.051948,2015.0223,2014,0
15034,0.042208,2014.1015,2008,0


In [128]:
h_yr_sort = h_yr.sort_values(by = ['yr_renovated', 'yr_built', 'date'],ascending=False).reset_index(drop=True)
h_yr_sort.head()

,price,date,yr_built,yr_renovated
0,0.107792,2015.0331,1968,2015
1,0.192857,2015.0506,1964,2015
2,0.105844,2015.0327,1962,2015
3,0.093117,2015.0406,1959,2015
4,0.113312,2015.0501,1956,2015
